<h1><center>Laboratorio 3: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:   Sasha Oyanadel
- Nombre de alumno 2:   Pablo Vergara


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/pablo30vll/Laboratorio3_MDS7202-)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle("/Users/locoplaya666/Desktop/labslaboratoriodatos/lab3/online_retail_data.pickle")
df_retail.head()

### 1. Función para explorar características [0.5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [0.3 puntos]
- Imprima un conteo de datos nulos por variable [0.2 puntos]

**Nota**: Para generar los gráficos no es obligatorio el uso de `plotly`, pero si es altamente recomendado. Pueden encontrar más información de esta librería en este [enlace](https://plotly.com/python/).

**Respuesta:**

In [ ]:
import plotly
import numpy as np

print("Plotly:", plotly.__version__)
print("Numpy:", np.__version__)

In [ ]:
def explore_data(df, nulls=False, columns=["Price", "Quantity"]):
    for col in columns:
        max_val = df[col].max()
        fig = px.histogram(df, x=col, nbins=35, title=f"Histograma de {col}", range_x=[0, max_val])
        fig.show()

    if nulls:
        print("Conteo de datos nulos por variable:")
        print(df[columns].isnull().sum())

explore_data(df_retail,nulls=True)


### 2. Eliminando outliers [1.0 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [0.5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

**Hint:** tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Nota:** No modificar el método set_output de la clase IQR

**Respuesta:**

In [ ]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, lam = 1.5):
    self.lam = lam
    self.q1 = None
    self.q2 = None
    self.q3 = None
    self.ric = None

  def fit(self, X):
    X= pd.DataFrame(X)
    self.q1 = X.quantile(0.25)
    self.q3 = X.quantile(0.75)
    self.ric = self.q3-self.q3
    return self

  def transform(self, X):
    X= pd.DataFrame(X)
    lower_bound = self.q1-self.ric*self.lam
    upper_bound = self.q3+self.ric*self.lam
    mask_list = []

    for idx, row in X.iterrows():
        es_valida = True
        for col in X.columns:
            if row[col] < lower_bound[col] or row[col] > upper_bound[col]:
                es_valida = False
                break  
        mask_list.append(es_valida)

    return X[mask_list]


  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [0.5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [0.1 puntos]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [0.1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [0.1 puntos]


**Hint:** El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.

**Nota:** Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
valores_lambda = [0.5, 1, 1.5, 2]

numerical_columns = ["Price", "Quantity"]
categorical_columns = ["Invoice", "StockCode", "Description", "InvoiceDate", "Customer ID", "Country"]

def explorar_con_lambda(lam):
    print(f"\n================ λ = {lam} ================\n")

    numeric_transformations = Pipeline([
        ("iqr_filter", IQR(lam=lam))
    ])

    column_transformer = ColumnTransformer([
        ("numerical", numeric_transformations, numerical_columns),
        ("categorical", "passthrough", categorical_columns)
    ], verbose_feature_names_out=False)

    column_transformer.set_output(transform="pandas")

    df_iqr = column_transformer.fit_transform(df_retail)


    print("\nDistribución de Price y Quantity con outliers eliminados:\n")
    explore_data(df_iqr)

for lam in valores_lambda:
    explorar_con_lambda(lam)


Al aplicar el filtro de outliers utilizando el método del rango intercuartil (IQR), se observa un cambio significativo en la distribución de las variables Price y Quantity. En los histogramas originales, sin filtrar, ambas variables presentan valores extremadamente altos que aplastan la visualización, haciendo que la mayor parte de los datos se concentren cerca de cero y ocultando su distribución real. Al aplicar el filtro IQR con distintos valores de lambda (λ = 0.5, 1, 1.5 y 2), los histogramas cambian drásticamente, mostrando una distribución mucho más clara y detallada, donde se identifican frecuencias típicas de precio y cantidad. Sin embargo, se observa que al variar lambda en ese rango, los gráficos resultantes no cambian significativamente. Esto ocurre porque los outliers en este dataset son valores extremadamente lejanos del centro de la distribución, por lo que quedan fuera del rango permitido incluso con valores de λ relativamente altos.

### 3. Agregando un imputer al pipeline [1.0 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [0.1 puntos]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [0.1 puntos]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [0.1 puntos]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [0.2 puntos]

**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
from sklearn.impute import SimpleImputer
categoric_transformations = Pipeline([
    ("mode_imputer", SimpleImputer(strategy="most_frequent"))
])

numeric_transformations = Pipeline([
    ("iqr_filter", IQR(lam=1.5)),
    ("mean_imputer", SimpleImputer(strategy="mean"))
])

numerical_columns = ["Price", "Quantity"]
categorical_columns = ["Invoice", "StockCode", "Description", "InvoiceDate", "Customer ID", "Country"]

column_transformer_mean = ColumnTransformer([
    ("numerical", numeric_transformations, numerical_columns),
    ("categorical", categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

column_transformer_mean.set_output(transform="pandas")

df_mean_imputer = column_transformer_mean.fit_transform(df_retail)

In [ ]:
explore_data(df_mean_imputer)

In [ ]:
from sklearn.impute import KNNImputer

numeric_transformations_knn = Pipeline([
    ("iqr_filter", IQR(lam=1.5)),
    ("knn_imputer", KNNImputer(n_neighbors=5))  
])

column_transformer_knn = ColumnTransformer([
    ("numerical", numeric_transformations_knn, numerical_columns),
    ("categorical", categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

column_transformer_knn.set_output(transform="pandas")

df_knn_imputer = column_transformer_knn.fit_transform(df_retail)


In [ ]:
explore_data(df_knn_imputer)

4) Al comparar df_mean_imputer y df_iqr, la variable Price mantiene una distribución muy similar, lo cual indica que la imputación por media no afectó significativamente esa variable. Sin embargo, en Quantity se observan aumentos notorios en valores frecuentes como 12, 6 y 2, lo que sugiere que el uso de la media generó concentraciones artificiales en ciertos valores. En resumen, la imputación por media puede mantener la forma general en algunas variables, pero distorsionar otras.

6) Al observar df_knn_imputer en comparación con df_iqr, se aprecia que la distribución en ambas variables es muy parecida a la original. Aunque algunos valores comunes como 12 siguen siendo frecuentes, no se exageran como con la media. La distribución es más suave y natural, especialmente en Quantity, lo que refleja que el método KNN imputa valores de manera más contextual y representativa.

7) Comparando df_knn_imputer y df_mean_imputer, se concluye que KNN es superior, especialmente en la variable Quantity. Mientras la media tiende a concentrar los datos en valores frecuentes, KNN mantiene una distribución más equilibrada, lo que es beneficioso para evitar sesgos. Por tanto, se escoge KNNImputer como método de imputación final, al preservar mejor la estructura estadística sin introducir distorsiones notables.



### 4. Creación de nuevas features [2.0 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [1.0 puntos]

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def get_LRMFP_features(df):
    df = df.copy()
    df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

    df["Total"] = df["Price"] * df["Quantity"]

    fecha_max = df["InvoiceDate"].max()

    agrupado = df.groupby("Customer ID")

    length = (agrupado["InvoiceDate"].max() - agrupado["InvoiceDate"].min()).dt.days

    recency = (fecha_max - agrupado["InvoiceDate"].max()).dt.days

    frequency = agrupado["Invoice"].nunique()

    monetary = agrupado["Total"].sum() / frequency

    df_sorted = df.sort_values(["Customer ID", "InvoiceDate"])
    df_sorted["IVT"] = df_sorted.groupby("Customer ID")["InvoiceDate"].diff().dt.days
    periodicity = df_sorted.groupby("Customer ID")["IVT"].std()

    lrmfp = pd.DataFrame({
        "Length": length,
        "Recency": recency,
        "Frequency": frequency,
        "Monetary": monetary,
        "Periodicity": periodicity
    })

    return lrmfp


In [ ]:
df_lrmfp = get_LRMFP_features(df_retail)
df_lrmfp.head()


#### 4.2 Agregando las custom features [1.0 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [0.1 puntos]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [0.1 puntos]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [0.5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [0.3 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [ ]:

def add_lrmfp_features(df):
    lrmfp = get_LRMFP_features(df)
    df = df.copy()
    df = df.drop_duplicates(subset="Customer ID")  
    df = df.merge(lrmfp, on="Customer ID", how="left")
    return df

custom_features = FunctionTransformer(add_lrmfp_features, validate=False)

In [ ]:

retail_pipeline = Pipeline([
    ("col_transformer", column_transformer_knn), 
    ("custom_features", custom_features)
])


In [ ]:
df_custom = retail_pipeline.fit_transform(df_retail)


In [ ]:
df_custom.head()

In [ ]:
explore_data(df_custom, columns=["Length", "Recency", "Monetary", "Frequency", "Periodicity"])


Al explorar la distribución de las nuevas variables LRMFP, se observa que:


- Length muestra una distribución con una gran cantidad de clientes con poca antigüedad y otro grupo con más de 250 días. Esto sugiere dos tipos de clientes: recientes y recurrentes.


- Recency tiene una distribución fuertemente sesgada a la derecha, indicando que la mayoría de los clientes han comprado recientemente. Es útil para identificar clientes activos frente a inactivos.


- Monetary presenta una distribución sesgada a la derecha, lo cual indica que la mayoría de los clientes hacen compras pequeñas, pero unos pocos realizan compras muy elevadas.


- Frequency  muestra una distribución muy concentrada en valores bajos (especialmente en 1), lo que indica que la mayoría de los clientes compra pocas veces, y muy pocos son compradores frecuentes.


- Periodicity también está sesgada a la derecha, lo cual significa que los clientes que repiten compras tienden a hacerlo con bastante separación temporal. La mediana parece estar por debajo de 20 días.




insight:
El análisis de las variables LRMFP indica que la mayoría de los clientes son compradores de bajo valor y con poca frecuencia, pero hay un grupo más reducido de clientes fieles y de alto gasto. Esto sugiere que el negocio puede beneficiarse significativamente si enfoca campañas de retención y fidelización hacia estos clientes frecuentes y de alto Monetary, ya que representan un alto valor con menor esfuerzo de adquisición.



### 5. MinMax Scaler [1.0 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [0.5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.min = None
        self.max = None

    def fit(self,X):
        X = pd.DataFrame(X)
        self.min = X.min()
        self.max = X.max()
        return self

    def transform(self,X):
        X = pd.DataFrame(X)
        return (X - self.min) / (self.max - self.min)

    def set_output(self,transform='default'):
        #No modificar este método
        return self



#### 5.2 Incorporando MinMax al pipeline [0.5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [0.1 puntos]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables.  [0.2 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
numeric_transformations = Pipeline([
    ("iqr_filter", IQR(lam=1.5)),
    ("imputer", SimpleImputer(strategy="mean")),  
    ("minmax", MinMax())
])


In [ ]:
column_transformer_minmax = ColumnTransformer([
    ("numerical", numeric_transformations, numerical_columns),
    ("categorical", categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

column_transformer_minmax.set_output(transform="pandas")

df_minmax = column_transformer_minmax.fit_transform(df_retail)


In [ ]:
print("Distribución en datos originales:")
explore_data(df_retail, columns=["Price", "Quantity"])

print("Distribución en datos escalados:")
explore_data(df_minmax, columns=["Price", "Quantity"])


Luego de aplicar la transformación MinMax, se observan cambios importantes en la escala de las variables numéricas. En el caso de Price, pasó de tener valores con una distribución altamente sesgada a la izquierda (con algunos valores extremos por sobre los 10.000) a estar contenida entre 0 y 1, donde la mayoría de los valores se concentra bajo 0.8. Esto evita que los valores extremos dominen la escala. En cuanto a Quantity, también se observaba una fuerte concentración en valores bajos (con outliers sobre 18.000). Tras la normalización, estos valores fueron mapeados al intervalo [0, 1], permitiendo una comparación más justa entre registros y eliminando el sesgo de escala. Ambos histogramas normalizados presentan una distribución más compacta y comparable entre observaciones.

### 6. Pregunta teórica [0.5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

La creación de pipelines en el proceso de Feature Engineering es útil porque permite automatizar, organizar y estandarizar todas las etapas de preprocesamiento de datos de manera estructurada. Con un pipeline, cada paso (como limpieza, imputación, escalamiento y generación de nuevas variables) se aplica secuencialmente y de forma reproducible, lo que reduce errores y asegura que los mismos procedimientos se apliquen tanto en los datos de entrenamiento como en datos futuros. En resumen, los pipelines hacen que el procesamiento de datos sea más limpio, confiable y escalable.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>